## TEST DATASET INITIALISATION

In [3]:
import sys
sys.path.append('../')

import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data_utils

import datasets
import importlib
importlib.reload(datasets)

<module 'datasets' from '/home/cagnetta/Projects/hierarchy-learning/datasets/../datasets/__init__.py'>

In [4]:
v = 16
m = 8
L = 2
n = 8
s = 2

input_size = s**L # number of pixels, actual input size is (input_size x num_features) because of one-hot encoding
num_data = n * (m**((s**L-1)/(s-1))) # total number of data
print(input_size, num_data)

4 4096.0


In [5]:
seed_rules = 12345678 # seed of the random hierarchy model
train_size = 1024 # size of the training set
test_size = 1024 # size of the test set

# to generate the full dataset: set trainset=num_data, test_size=0

dataset = datasets.RandomHierarchyModel(
    num_features=v, # vocabulary size
    num_synonyms=m, # features multiplicity
    num_layers=L, # number of layers
    num_classes=n, # number of classes
    tuple_size=s, # number of branches of the tree
    seed_rules=seed_rules,
    train_size=train_size,
    test_size=test_size,
    input_format='onehot',
    whitening=1 # 1 to whiten the input
)

In [8]:
x = dataset.features
print(x.size())
print(x.mean(dim=1).mean())
print(x.norm(dim=1).mean())

torch.Size([2048, 16, 4])
tensor(-2.7603e-10)
tensor(1.0000)


In [9]:
print(dir(dataset)) 

# for the input points call trainset.input
print(dataset.features.size()) # dimension: train_size x num_features x input_size

# for the labels call trainset.output
print(dataset.labels.size()) # dimension: train_size

['__add__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__orig_bases__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_is_protocol', 'features', 'labels', 'num_classes', 'num_features', 'num_layers', 'num_synonyms', 'transform', 'tuple_size']
torch.Size([2048, 16, 4])
torch.Size([2048])


In [10]:
print(dataset.features[0,:,:])

tensor([[-0.0645, -0.0645, -0.0645, -0.0645],
        [-0.0645, -0.0645, -0.0645, -0.0645],
        [-0.0645, -0.0645, -0.0645, -0.0645],
        [ 0.9682, -0.0645, -0.0645, -0.0645],
        [-0.0645, -0.0645, -0.0645, -0.0645],
        [-0.0645, -0.0645, -0.0645, -0.0645],
        [-0.0645, -0.0645, -0.0645,  0.9682],
        [-0.0645, -0.0645, -0.0645, -0.0645],
        [-0.0645, -0.0645, -0.0645, -0.0645],
        [-0.0645, -0.0645, -0.0645, -0.0645],
        [-0.0645,  0.9682, -0.0645, -0.0645],
        [-0.0645, -0.0645,  0.9682, -0.0645],
        [-0.0645, -0.0645, -0.0645, -0.0645],
        [-0.0645, -0.0645, -0.0645, -0.0645],
        [-0.0645, -0.0645, -0.0645, -0.0645],
        [-0.0645, -0.0645, -0.0645, -0.0645]])
